Classified the well logs we have with model we have trained an evaluate

Import necessary libraries and dataset we want to classified

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('IMPUTED DATA/7_1-1_imputed.csv') #Adjust with well imputed file

df.head()
df.info()

Seperate the depth and feature values that used for predict the lithlogy

In [ ]:
filtered_model = ['GR', 'RHOB', 'NPHI', 'DTC', 'RDEP', 'RXO', 'RSHA', 'FORCE_2020_LITHOFACIES_LITHOLOGY'] 
df_filtered_model = df[filtered_model]
other_columns = df.drop(columns=filtered_model)
other_columns.info()
df_filtered_model.info()
df_filtered_model_classified = df_filtered_model.copy()

Load the model to predict the lithology on well logs data

In [ ]:
import joblib

model = joblib.load('lithology_classifier.pkl')

# Filter rows where the target column has missing values
missing_mask = df_filtered_model_classified['FORCE_2020_LITHOFACIES_LITHOLOGY'].isna()
df_missing = df_filtered_model_classified[missing_mask]

# Drop the target column to get features
X_missing = df_missing.drop(columns=['FORCE_2020_LITHOFACIES_LITHOLOGY'])

# Predict missing values
if not X_missing.empty:
    predicted_values = model.predict(X_missing)

    # Update the DataFrame with the predicted values
    df_filtered_model_classified.loc[missing_mask, 'FORCE_2020_LITHOFACIES_LITHOLOGY'] = predicted_values

    # Save the updated DataFrame to a CSV file
    df_filtered_model_classified.to_csv('7_1-1_Classified.csv', index=False)

    print("Missing values predicted and saved to '7_1-1_Classified.csv'.")
else:
    print("No missing values found to predict.")

Combine the predicted lithology dataset with depth value again to see wich lithology on wich depth

In [ ]:
df_filtered_model_classified['FORCE_2020_LITHOFACIES_LITHOLOGY'].unique()
df_filtered_model_classified.info()
df_result = pd.concat([other_columns, df_filtered_model_classified], axis=1)
df_result.info()

defince the mapping lithology based on FORCE website with predicted values and import the result to csv

In [ ]:
# Define the mapping dictionary
lithology_mapping = {
    30000: 'Sandstone',
    65030: 'Sandstone/Shale',
    65000: 'Shale',
    80000: 'Marl',
    74000: 'Dolomite',
    70000: 'Limestone',
    70032: 'Chalk',
    88000: 'Halite',
    86000: 'Anhydrite',
    99000: 'Tuff',
    90000: 'Coal',
    93000: 'Basement'
}

# Replace the values in the 'FORCE_2020_LITHOFACIES_LITHOLOGY' column using the mapping
df_result['FORCE_2020_LITHOFACIES_LITHOLOGY'] = df_result['FORCE_2020_LITHOFACIES_LITHOLOGY'].replace(lithology_mapping)
df_result.to_csv('7_1-1_Lithology.csv', index=False)

In [ ]:
df_result['FORCE_2020_LITHOFACIES_LITHOLOGY'].unique()